<a href="https://colab.research.google.com/github/iamanujkumar/Natural-Language-Processing/blob/main/YouTube_ChatBot_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEndpointEmbeddings

In [ ]:
import os
from google.colab import userdata

**Indexing**

In [ ]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 8.6 MB/s eta 0:00:00


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [ ]:
pip install youtube-transcript-api

In [ ]:
ytt_api = YouTubeTranscriptApi()
fetched_transcript = ytt_api.fetch(video_id)
for snippet in fetched_transcript:
    print(snippet.text)

last_snippet = fetched_transcript[-1]

snippet_count = len(fetched_transcript)

Imagine you happen across a short movie script that
describes a scene between a person and their AI assistant.
The script has what the person asks the AI, but the AI's response has been torn off.
Suppose you also have this powerful magical machine that can take
any text and provide a sensible prediction of what word comes next.
You could then finish the script by feeding in what you have to the machine,
seeing what it would predict to start the AI's answer,
and then repeating this over and over with a growing script completing the dialogue.
When you interact with a chatbot, this is exactly what's happening.
A large language model is a sophisticated mathematical function
that predicts what word comes next for any piece of text.
Instead of predicting one word with certainty, though,
what it does is assign a probability to all possible next words.
To build a chatbot, you lay out some text that describes an interaction between a user
and a hypothetical AI assistant, add on whatever the use

In [ ]:
fetched_transcript

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text='Imagine you happen across a short movie script that', start=1.14, duration=2.836), FetchedTranscriptSnippet(text='describes a scene between a person and their AI assistant.', start=3.976, duration=3.164), FetchedTranscriptSnippet(text="The script has what the person asks the AI, but the AI's response has been torn off.", start=7.48, duration=5.58), FetchedTranscriptSnippet(text='Suppose you also have this powerful magical machine that can take', start=13.06, duration=3.92), FetchedTranscriptSnippet(text='any text and provide a sensible prediction of what word comes next.', start=16.98, duration=3.98), FetchedTranscriptSnippet(text='You could then finish the script by feeding in what you have to the machine,', start=21.5, duration=4.006), FetchedTranscriptSnippet(text="seeing what it would predict to start the AI's answer,", start=25.506, duration=2.862), FetchedTranscriptSnippet(text='and then repeating this over and over with 

In [ ]:
transcript = " ".join(snippet.text for snippet in fetched_transcript)
print(transcript)

Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical AI assistant, add on whatever the use

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunk = splitter.create_documents([transcript])

In [ ]:
len(chunk)

10

**Embedding**

In [ ]:
from google.colab import userdata
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HUGGINGFACEHUB_API_TOKEN")

embedding_model = HuggingFaceEndpointEmbeddings(
    repo_id="sentence-transformers/all-MiniLM-L6-v2",
)

In [ ]:
vector_store = FAISS.from_documents(chunk, embedding_model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
vector_store.index_to_docstore_id

{0: '23675828-81d5-443c-8916-6662827dc6c6',
 1: 'c6b3ad80-c4f8-4780-a8d7-6f0f6ebbe5d1',
 2: 'e79b4236-d3fa-4847-8d6f-4a7611bc75fe',
 3: '658dc7b9-c2dd-4671-91dd-a7d776eb2c22',
 4: '2bb74dd1-aece-4b38-89c0-cf49ef99b055',
 5: '5e424f33-7d7b-4ec5-974c-137ed62580fc',
 6: '61c10c32-b278-423a-9834-e09b1f0462d9',
 7: '38fac04c-16fb-4c7c-ab51-d2bd231bcd45',
 8: '955baa29-9723-4ee2-add0-1d7ede042477',
 9: '479115f0-3d57-472b-8b01-3df1423ca6e4'}

In [ ]:
vector_store.get_by_ids(['23675828-81d5-443c-8916-6662827dc6c6'])

[Document(id='23675828-81d5-443c-8916-6662827dc6c6', metadata={}, page_content="Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an int

**Retriever**

In [79]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 6})

In [ ]:
retriever.invoke('what is llm')

[Document(id='38fac04c-16fb-4c7c-ab51-d2bd231bcd45', metadata={}, page_content="type of operation known as a feed-forward neural network, and this gives the model extra capacity to store more patterns about language learned during training. All of this data repeatedly flows through many different iterations of these two fundamental operations, and as it does so, the hope is that each list of numbers is enriched to encode whatever information might be needed to make an accurate prediction of what word follows in the passage. At the end, one final function is performed on the last vector in this sequence, which now has had a chance to be influenced by all the other context from the input text, as well as everything the model learned during training, to produce a prediction of the next word. Again, the model's prediction looks like a probability for every possible next word. Although researchers design the framework for how each of these steps work, it's important to understand that the s

**Augmentation**

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    groq_api_key=userdata.get("GROQ_API_KEY"),
    model="llama-3.1-8b-instant",
    temperature=0,
    )

In [ ]:
prompt = PromptTemplate(
    template = """
    You are a helpful assistant
    Answer ONLY from the provided transcript context.
    If the context is insufficient, say "I don't know".

    {context}

    Question: {question}

    """,
    input_variables=['context', 'question']
)

In [89]:
question = "In this video discuss about LLM"
retrieved_docs = retriever.invoke(question)

In [90]:
retrieved_docs

[Document(id='479115f0-3d57-472b-8b01-3df1423ca6e4', metadata={}, page_content='the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.'),
 Document(id='38fac04c-16fb-4c7c-ab51-d2bd231bcd45', metadata={}, page_content="type of operation known as a feed-forward neural network, and this gives the model extra capacity to store more patterns about language learned during training. All of this data repeatedly flows through many different iterations of these two fundamental operations, and as it does so, the hope is that each list of numbers is enriched to encode whatever information might be needed to make an accurate prediction of what word follows in the passage. At the end, one final function is performed on the last vec

In [91]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.\n\ntype of operation known as a feed-forward neural network, and this gives the model extra capacity to store more patterns about language learned during training. All of this data repeatedly flows through many different iterations of these two fundamental operations, and as it does so, the hope is that each list of numbers is enriched to encode whatever information might be needed to make an accurate prediction of what word follows in the passage. At the end, one final function is performed on the last vector in this sequence, which now has had a chance to be influenced by all the other context from the input text, as well as everything the model learned during 

In [92]:
final_prompt = prompt.invoke({'context': context_text, 'question': question})

In [93]:
final_prompt

StringPromptValue(text='\n    You are a helpful assistant\n    Answer ONLY from the provided transcript context.\n    If the context is insufficient, say "I don\'t know".\n\n    the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.\n\ntype of operation known as a feed-forward neural network, and this gives the model extra capacity to store more patterns about language learned during training. All of this data repeatedly flows through many different iterations of these two fundamental operations, and as it does so, the hope is that each list of numbers is enriched to encode whatever information might be needed to make an accurate prediction of what word follows in the passage. At the end, one final function is perform

**Generation**

In [94]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, in this video, the speaker discusses about Large Language Models (LLM).


**Chain**

In [95]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [96]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text


In [97]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [98]:
parallel_chain.invoke('What is llm')

{'context': "type of operation known as a feed-forward neural network, and this gives the model extra capacity to store more patterns about language learned during training. All of this data repeatedly flows through many different iterations of these two fundamental operations, and as it does so, the hope is that each list of numbers is enriched to encode whatever information might be needed to make an accurate prediction of what word follows in the passage. At the end, one final function is performed on the last vector in this sequence, which now has had a chance to be influenced by all the other context from the input text, as well as everything the model learned during training, to produce a prediction of the next word. Again, the model's prediction looks like a probability for every possible next word. Although researchers design the framework for how each of these steps work, it's important to understand that the specific behavior is an emergent phenomenon based on how those hundr

In [99]:
parser = StrOutputParser()

In [100]:
main_chain = parallel_chain | prompt | llm | parser

In [103]:
main_chain.invoke('what is llm')

'A large language model (LLM) is a sophisticated mathematical function that predicts what word comes next for any piece of text.'